In [56]:
# These dependencies are necessary for loading the data
import json
import os
import random
import numpy as np

In [57]:
#This will be moved down later to empasize flow better
#user inputs:
user_genre_1 = 'Alternative/Indie'
user_genre_2 = 'Pop rock'
user_language = 'English'
user_tempo = '120'
user_playlist = json.load(open('Playlist.json'))

TRAINING OUR MODEL

We will be doing a supervised classification model with random forest. Each tree will produce a prediction if a song satisfies the inputs. The inputs will be randomly selected. 

Each decision in the decision tree will be given a weight which will start with 1 and will be adjusted according to feedback. From here we will calculate the average of each decision tree output and compare it to a threshold of accpetance. If the average if greater than the threshold, the song is labeled as in the new generated playlist. If the average is bellow, the song is labeled as not part of the new playlist. 

In [58]:
#SETUP
#our training data
music_train = json.load(open('Song_Dataset.json'))
music_comp1 = json.load(open('Genre_Train1.json')) #genres alternative/indie and pop


#the score for each song in the decision trees
score = 0


#our weights the perceptron model performed after the random forest produces a score
w = 1;
b = 0;


#list of music genres
categories = json.load(open('catergories.json'))
c_len = len(categories["categories"])


#list of languages
languages = json.load(open('languages.json'))
l_len = len(languages["languages"])


#randomly choose the inputs
genre1 = categories["categories"][random.randrange(c_len)]["c"]
genre2 = categories["categories"][random.randrange(c_len)]["c"]
language = languages["languages"][random.randrange(l_len)]["l"]
tempo = random.randint(50, 200)

The function decision_tree takes in 8 inputs:

our taining set, genre1, genre2, language, tempo, weight for the 1st decision tree, weight for the 2nd decision tree, weight for the 3rd decision tree

The function then outputs an array of predicted songs based on if they are within the threshold. The threshold is determined via the weights of each decision tree.

In [59]:
#DECISION TREES
def decision_tree(music, genre1, genre2, language, tempo, w):
    p1 = np.empty((len(music["songs"]), 2))
    p2 = np.empty((len(music["songs"]), 2))
    p3 = np.empty((len(music["songs"]), 2))

    
    
    #decision tree 1: starting with genres
    for i in range (0 , len(music["songs"])):
        score = 0   #reset the score for each song
        if ((music["songs"][i]["genre1"] == genre1) | (music["songs"][i]["genre1"] == genre2) | (music["songs"][i]["genre2"] == genre2) | (music["songs"][i]["genre2"] == genre1)):
                score += 1
                if (music["songs"][i]["language"] == language):
                    score += 1
                    if ((int(music["songs"][i]["tempo"]) > (tempo - 25)) & (int(music["songs"][i]["tempo"]) < (tempo + 25))):
                        score += 1
                        p1[i][0] = i
                        p1[i][1] = score
                    else:
                        p1[i][0] = i
                        p1[i][1] = score
                else:
                    p1[i][0] = i
                    p1[i][1] = score
            else:
                p1[i][0] = i
                p1[i][1] = score



    #decision tree 2: starting with language
    for i in range (0 , len(music["songs"])):
        score = 0   #reset the score for each song
        if (music["songs"][i]["language"] == language):
            score += 1
            if ((music["songs"][i]["genre1"] == genre1) | (music["songs"][i]["genre2"] == genre2) | (music["songs"][i]["genre1"] == genre2) | (music["songs"][i]["genre2"] == genre1)):
                score += 1
                if ((int(music["songs"][i]["tempo"]) > (tempo - 25)) & (int(music["songs"][i]["tempo"]) < (tempo + 25))):
                    score += 1
                    p2[i][0] = i
                    p2[i][1] = score
                else:
                    p2[i][0] = i
                    p2[i][1] = score
            else:
                p2[i][0] = i
                p2[i][1] = score
        else:
            p2[i][0] = i
            p2[i][1] = score



    #decision tree 3: starting with tempo
    for i in range (0 , len(music["songs"])):
        score = 0   #reset the score for each song
        if ((int(music["songs"][i]["tempo"]) > (tempo - 25)) & (int(music["songs"][i]["tempo"]) < (tempo + 25))):
            score += 1
            if (music["songs"][i]["language"] == language):
                score += 1
                if ((music["songs"][i]["genre1"] == genre1) | (music["songs"][i]["genre2"] == genre2) | (music["songs"][i]["genre1"] == genre2) | (music["songs"][i]["genre2"] == genre1)):
                    score += 1
                    p3[i][0] = i
                    p3[i][1] = score
                else:
                    p3[i][0] = i
                    p3[i][1] = score
            else:
                p2[i][0] = i
                p3[i][1] = score
        else:
            p3[i][0] = i
            p3[i][1] = score
        
        
    #calculate the total score of each song based on the three trees
    total_scores = np.zeros((len(music["songs"]), 2))
    
    j = 0   #counter to add results to y_hat 
    
    for i in range (0 , len(music["songs"])):
        predict = w * ((p1[i][1] + p2[i][1] + p3[i][1]) / 3) + b
        
        #threshold is 2 for now
        ############### need to find a way to calculate this ##############
        if(predict > threshold):
            total_scores[j][0] = i
            total_scores[j][1] = predict
            j += 1
        else:
            #this is to size the array appropiately to delete unnecessary zeros. 
            total_scores = np.delete(total_scores, len(total_scores)-1, 0)
    return total_scores
        
    

Call the decision_tree function over while loop to train the our model by updating the weights for each of the decision trees.

In [60]:
music_scores = decision_tree(music_train, genre1, genre2, language, tempo, w1, w2, w3, b1, b2, b3)

In [1]:
###fix
def train (music_train, music_comp, genre1, genre2, language, tempo, w1, w2, w3, b1, b2, b3):
    
    a = 0.1
    accuracy = np.zeros()
    
    while ((accuracy / len(music_comp["songs"])) < 0.5 ):
        music_scores = decision_tree(music_train, genre1, genre2, language, tempo, w1, w2, w3, b1, b2, b3)
        
        for i in music_scores[:][0]:
            for j in range (0 , len(music_comp["songs"])):
                if((music_train["songs"][i]["name"] == music_comp["songs"][j]["name"]) | (music_train["songs"][i]["name"] == music_comp["songs"][j]["name"]) | (music_train["songs"][i]["name"] == music_comp["songs"][j]["name"]) | (music_train["songs"][i]["name"] == music_comp["songs"][j]["name"])):
                    accuracy1 += 1
                    print (i);
                    
                    
        if((accuracy1 / len(music_comp["songs"])) < 0.5):
            w1 = w1 - a(y_hat - y)x
            b1 = b1 - a(y_hat - y)


SyntaxError: invalid syntax (3252770278.py, line 17)

'Korean Rock/Alternative'

In [64]:


w = w - a(y_hat - y)x
b = b - a(y_hat - y)

SyntaxError: invalid syntax (585545993.py, line 1)